In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   validation_split=0.2)

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
train_generator = train_datagen.flow_from_directory(
        '../data/archive/train/eyes',  # This is the source directory for training images
        target_size=(80, 80),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16,
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        '../data/archive/train/eyes',  # This is the source directory for training images
        target_size=(80, 80),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16,
        subset='validation')

Found 1162 images belonging to 2 classes.
Found 290 images belonging to 2 classes.


In [5]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    include_top=False, weights='imagenet', input_shape=(80,80,3)
)

In [6]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [7]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [8]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
      train_generator,
      validation_data = validation_generator,
      epochs=50,
      verbose=1,
      callbacks=early_stop)

Epoch 1/50
73/73 [==============================] - 17s 104ms/step - loss: 0.1259 - accuracy: 0.9535 - val_loss: 2.5076 - val_accuracy: 0.9069
Epoch 2/50
73/73 [==============================] - 7s 95ms/step - loss: 0.0851 - accuracy: 0.9742 - val_loss: 4.2266 - val_accuracy: 0.9034
Epoch 3/50
73/73 [==============================] - 6s 88ms/step - loss: 0.0394 - accuracy: 0.9931 - val_loss: 1.5392 - val_accuracy: 0.9552
Epoch 4/50
73/73 [==============================] - 5s 67ms/step - loss: 0.0391 - accuracy: 0.9940 - val_loss: 0.8952 - val_accuracy: 0.9690
Epoch 5/50
73/73 [==============================] - 5s 69ms/step - loss: 0.0346 - accuracy: 0.9914 - val_loss: 1.6421 - val_accuracy: 0.9414
Epoch 6/50
73/73 [==============================] - 6s 87ms/step - loss: 0.0272 - accuracy: 0.9914 - val_loss: 0.5111 - val_accuracy: 0.9793
Epoch 7/50
73/73 [==============================] - 6s 76ms/step - loss: 0.0027 - accuracy: 0.9991 - val_loss: 0.4625 - val_accuracy: 0.9862
Epoch 8/50


In [12]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
test_generator = test_datagen.flow_from_directory(
        '../data/test/yourfolderforcroppedeyes',  # This is the source directory for training images
        target_size=(80, 80),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

Found 45 images belonging to 2 classes.


In [14]:
model.evaluate(test_generator)

3/3 [==============================] - 0s 23ms/step - loss: 0.8695 - accuracy: 0.9556


[0.8694506883621216, 0.9555555582046509]

In [15]:
model.save("mobile_eyes_model.h5")